# <font color="#418FDE" size="6.5" uppercase>**TF 2.20 Overview**</font>

>Last update: 20260125.
    
By the end of this Lecture, you will be able to:
- Summarize the core architectural concepts behind TensorFlow 2.x and eager execution. 
- Identify the primary high-level APIs in TensorFlow 2.20.0 and their roles. 
- Distinguish between stable, legacy, and experimental TensorFlow APIs relevant in 2026. 


## **1. Eager Execution Basics**

### **1.1. Immediate Tensor Execution**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_C/image_01_01.jpg?v=1769364297" width="250">



>* Ops run immediately and return real values
>* Feels like normal numerical coding, easier reasoning

>* Eager results enable natural debugging with Python tools
>* Supports fast, interactive experimentation, especially in notebooks

>* Eager mode hides device placement and optimizations
>* Pythonic interface atop scalable, graph-based runtime



In [ ]:
#@title Python Code - Immediate Tensor Execution

# This script demonstrates immediate tensor execution basics.
# We use only Python standard library for clarity.
# Focus on simple numeric operations and observations.

# Import the math module for numeric comparisons.
import math

# Define a helper function to print section titles.
def print_title(title):
    print("\n" + title)

# Show a simple numeric operation executing immediately.
print_title("1) Immediate numeric operations in Python")
result_add = 3 + 4
print("Addition result is", result_add)

# Show that the result is a concrete Python integer.
print("Type of result_add is", type(result_add))

# Demonstrate a small list comprehension executing immediately.
print_title("2) Immediate list comprehension execution")
values = [x * 2 for x in range(3)]
print("Doubled values are", values)

# Confirm the length to emphasize concrete data availability.
print("Length of values is", len(values))

# Define a function that mimics a tiny computation step.
def tiny_step(x):
    return x * x + 1

# Use a loop to show step by step immediate results.
print_title("3) Step by step loop execution")
current = 0
for step in range(3):
    current = tiny_step(current)
    print("After step", step, "value is", current)

# Demonstrate a conditional depending on an immediate result.
print_title("4) Condition based on immediate result")
threshold = 5
if current > threshold:
    print("Current is above threshold", threshold)
else:
    print("Current is not above threshold", threshold)

# Show a small numeric check using math for completeness.
print_title("5) Simple numeric sanity check")
root_two_squared = math.sqrt(2) ** 2
print("Squared root two approximately", round(root_two_squared, 3))




### **1.2. Graph Model Changes**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_C/image_01_02.jpg?v=1769364326" width="250">



>* TF2 runs ops eagerly, like NumPy
>* Graphs remain optional for optimization and deployment

>* Graphs shift from manual centerpiece to byproduct
>* TensorFlow compiles Python code into optimized deployable graphs

>* Eager mode simplifies debugging and Python control flow
>* Graphs optimize execution for accelerators and deployment



In [ ]:
#@title Python Code - Graph Model Changes

# This script illustrates TensorFlow graph model changes.
# It compares eager execution with a traced graph function.
# Focus on small tensors and clear printed outputs.

# !pip install tensorflow==2.20.0.

# Import TensorFlow with a clear alias.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Define a tiny helper to show a separator.
def print_sep(title):
    print("\n---", title, "---")

# Create a small constant tensor for demonstrations.
with tf.device("CPU:0"):
    x = tf.constant([1.0, 2.0, 3.0])

# Verify tensor shape is as expected.
assert x.shape == (3,), "Unexpected tensor shape for x"

# Define a simple Python function using TensorFlow ops.
def simple_compute(t):
    y = t * 2.0
    z = y + 1.0
    return z

# Show eager execution behavior with direct function call.
print_sep("Eager execution result")

# Call the function normally in eager mode.
result_eager = simple_compute(x)

# Print the concrete tensor values from eager execution.
print("Eager result:", result_eager.numpy())

# Use tf.function to trace the same Python function.
@tf.function
def simple_compute_graph(t):
    y = t * 2.0
    z = y + 1.0
    return z

# Show graph execution behavior using tf.function.
print_sep("Graph execution result")

# Call the traced function, which builds a graph internally.
result_graph = simple_compute_graph(x)

# Print the result, which matches eager computation.
print("Graph result:", result_graph.numpy())

# Inspect the concrete function to emphasize graph creation.
concrete = simple_compute_graph.get_concrete_function(
    tf.TensorSpec(shape=(3,), dtype=tf.float32)
)

# Show that a graph object exists behind the scenes.
print_sep("Graph object summary")

# Print a short description of the internal graph.
print("Graph type:", type(concrete.graph).__name__)

# Confirm that eager and graph results are identical.
print_sep("Consistency check")

# Use TensorFlow assertion to validate equality.
with tf.device("CPU:0"):
    tf.debugging.assert_near(result_eager, result_graph)

# Print final confirmation message for the learner.
print("Eager and graph computations match exactly.")



### **1.3. Tracing with tf function**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_C/image_01_03.jpg?v=1769364412" width="250">



>* tf.function traces eager code into graphs
>* Cached graphs run faster with graph optimizations

>* Tracing builds a symbolic graph from functions
>* Optimized graphs run faster and more consistently

>* Tracing builds input-specific, optimized computation graphs
>* Python side effects change, but deployment performance improves



In [ ]:
#@title Python Code - Tracing with tf function

# This script demonstrates TensorFlow eager tracing.
# It focuses on tf function and graphs.
# Run cells to observe tracing behavior.

# !pip install tensorflow==2.20.0.

# Import standard library modules safely.
import os
import random
import math

# Disable GPU to avoid CUDA errors in environments without valid handles.
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Set deterministic random seeds for reproducibility.
random.seed(7)

# Try importing TensorFlow and handle failures.
try:
    import tensorflow as tf
except ImportError as e:
    raise SystemExit("TensorFlow is required for this demo.")

# Print TensorFlow version in one concise line.
print("TensorFlow version:", tf.__version__)

# Check for available GPU devices briefly.
gpus = tf.config.list_physical_devices("GPU")

# Print whether a GPU is available for execution.
print("GPU available:", bool(gpus))

# Create two small constant tensors for demonstration.
a = tf.constant([1.0, 2.0, 3.0], dtype=tf.float32)

# Create another tensor with matching shape and type.
b = tf.constant([4.0, 5.0, 6.0], dtype=tf.float32)

# Define a simple eager function using TensorFlow ops.
def eager_add(x, y):
    return x + y

# Call the eager function and print the result.
result_eager = eager_add(a, b)

# Show that eager execution returns an immediate tensor.
print("Eager result:", result_eager.numpy())

# Decorate a function with tf function for tracing.
@tf.function
def traced_add(x, y):
    return x + y

# Call the traced function once to trigger tracing.
result_traced_first = traced_add(a, b)

# Print the first traced call result for comparison.
print("First traced result:", result_traced_first.numpy())

# Call the traced function again with same signature.
result_traced_second = traced_add(a, b)

# Print the second traced call result to show reuse.
print("Second traced result:", result_traced_second.numpy())

# Inspect the concrete function created by tracing.
concrete = traced_add.get_concrete_function(
    tf.TensorSpec(shape=(3,), dtype=tf.float32),
    tf.TensorSpec(shape=(3,), dtype=tf.float32),
)

# Print a short summary of the traced graph.
print("Concrete function inputs:", concrete.inputs)

# Print the output specification of the traced graph.
print("Concrete function output:", concrete.output_shapes)

# Define a function with simple TensorFlow control flow.
@tf.function
def traced_square_if_positive(x):
    return tf.cond(x > 0.0, lambda: x * x, lambda: -x)

# Call the control flow function with positive input.
positive_input = tf.constant(2.0, dtype=tf.float32)

# Call the function and capture the positive result.
positive_result = traced_square_if_positive(positive_input)

# Call the control flow function with negative input.
negative_input = tf.constant(-2.0, dtype=tf.float32)

# Call the function and capture the negative result.
negative_result = traced_square_if_positive(negative_input)

# Print both results to show consistent traced behavior.
print("Control flow results:", positive_result.numpy(), negative_result.numpy())

# End by confirming script finished without runtime errors.
print("Tracing demo completed successfully.")



## **2. Core High Level APIs**

### **2.1. Building Models With Keras**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_C/image_02_01.jpg?v=1769364488" width="250">



>* Keras is TensorFlow’s main high-level interface
>* Supports simple to complex models, hiding low-level details

>* Unified setup for compile, train, evaluate workflows
>* Same interface supports many different ML tasks

>* Keras balances simplicity with deep customization options
>* Acts as central hub connecting models, data, deployment



In [ ]:
#@title Python Code - Building Models With Keras

# This script shows Keras model building basics.
# It uses TensorFlow Keras for simple classification.
# Focus on defining compiling and training models.

# !pip install tensorflow==2.20.0.

# Import standard library modules safely.
import os
import random
import math

# Set deterministic random seeds for reproducibility.
random.seed(7)

# Try importing TensorFlow and handle absence.
try:
    import tensorflow as tf
except ImportError as e:
    raise SystemExit("TensorFlow is required for this demo.")

# Disable GPU to avoid CUDA errors in environments without proper GPU setup.
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Select device based on GPU availability.
physical_gpus = tf.config.list_physical_devices("GPU")
if physical_gpus:
    device_type = "GPU"
else:
    device_type = "CPU"

# Print which device type will be used.
print("Using device type:", device_type)

# Load MNIST dataset from Keras utilities.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Confirm dataset shapes before preprocessing.
print("Train shape:", x_train.shape, "Test shape:", x_test.shape)

# Use a small subset for quick demonstration.
subset_size = 2000
x_train_small = x_train[:subset_size]
y_train_small = y_train[:subset_size]

# Normalize pixel values to range zero one.
x_train_small = x_train_small.astype("float32") / 255.0
x_test_small = x_test.astype("float32") / 255.0

# Flatten images to vectors for dense network.
input_shape = (28 * 28,)
x_train_small = x_train_small.reshape((-1,) + input_shape)
x_test_small = x_test_small.reshape((-1,) + input_shape)

# Validate shapes after reshaping operation.
assert x_train_small.shape[1:] == input_shape
assert x_test_small.shape[1:] == input_shape

# Build a simple Sequential Keras model.
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax"),
])

# Show a short model summary using print.
model.summary(print_fn=lambda line: print(line))

# Compile model with optimizer loss and metrics.
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# Train model briefly with silent verbose setting.
history = model.fit(
    x_train_small,
    y_train_small,
    epochs=3,
    batch_size=64,
    verbose=0,
    validation_split=0.2,
)

# Evaluate model on a small test subset.
loss, accuracy = model.evaluate(
    x_test_small[:1000],
    y_test[:1000],
    verbose=0,
)

# Print concise training and evaluation results.
print("Final training accuracy:", round(history.history["accuracy"][-1], 4))
print("Validation accuracy:", round(history.history["val_accuracy"][-1], 4))
print("Test accuracy on subset:", round(accuracy, 4))



### **2.2. Efficient Data Pipelines**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_C/image_02_02.jpg?v=1769364561" width="250">



>* tf.data builds scalable, integrated Dataset pipelines
>* Automates loading, transforming, batching, and prefetching efficiently

>* Handles many data sources as Datasets
>* Runs complex preprocessing efficiently for smooth training

>* tf.data scales cleanly across devices and workers
>* Sharding, caching, prefetching keep training fast, balanced



In [ ]:
#@title Python Code - Efficient Data Pipelines

# This script introduces efficient TensorFlow data pipelines.
# We simulate tf.data ideas using pure Python lists.
# Focus on batching, shuffling, mapping, and prefetching.

# Required external libraries would be installed here.
# !pip install tensorflow.

# Import standard modules for randomness and typing.
import random
import itertools

# Set deterministic random seed for reproducible behavior.
random.seed(42)

# Create a tiny synthetic dataset of feature label pairs.
raw_features = list(range(20))
raw_labels = [x % 2 for x in raw_features]

# Check dataset sizes before building the pipeline.
assert len(raw_features) == len(raw_labels)

# Define a simple normalization mapping function.

def normalize_feature(x):
    return (x - 10) / 10

# Define a function that yields shuffled indices.
def shuffled_indices(n_items, seed):
    indices = list(range(n_items))
    random.Random(seed).shuffle(indices)
    return indices

# Define a generator that yields mapped and shuffled samples.
def sample_generator(features, labels, seed):
    n_items = len(features)
    for idx in shuffled_indices(n_items, seed):
        feat = normalize_feature(features[idx])
        lab = labels[idx]
        yield feat, lab

# Define a batching function that groups samples.
def batch_generator(sample_iter, batch_size):
    batch_feats, batch_labs = [], []
    for feat, lab in sample_iter:
        batch_feats.append(feat)
        batch_labs.append(lab)
        if len(batch_feats) == batch_size:
            yield batch_feats, batch_labs
            batch_feats, batch_labs = [], []
    if batch_feats:
        yield batch_feats, batch_labs

# Define a simple prefetch wrapper using an internal buffer.
def prefetch_generator(batched_iter, buffer_size):
    buffer = []
    for batch in batched_iter:
        buffer.append(batch)
        if len(buffer) > buffer_size:
            yield buffer.pop(0)
    for batch in buffer:
        yield batch

# Build the pipeline step by step for one small epoch.
base_iter = sample_generator(raw_features, raw_labels, seed=123)

# Apply batching to the shuffled and normalized samples.
batched_iter = batch_generator(base_iter, batch_size=4)

# Apply prefetching to simulate overlapping work.
prefetched_iter = prefetch_generator(batched_iter, buffer_size=2)

# Collect a few batches to inspect the pipeline output.
collected_batches = list(itertools.islice(prefetched_iter, 3))

# Print a short summary of the pipeline behavior.
print("Total raw samples:", len(raw_features))
print("Number of collected batches:", len(collected_batches))
print("Batch shapes (features, labels) preview:")
for i, (bf, bl) in enumerate(collected_batches):
    print("Batch", i, "sizes:", len(bf), len(bl))

# Show actual normalized feature values for the first batch.
first_features, first_labels = collected_batches[0]
print("First batch features:", first_features)
print("First batch labels:", first_labels)

# Confirm that all labels are still correctly aligned.
reconstructed = []
for feats, labs in collected_batches:
    reconstructed.extend(list(zip(feats, labs)))
print("Reconstructed pairs count:", len(reconstructed))




### **2.3. Distributed Training Strategies**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_C/image_02_03.jpg?v=1769364635" width="250">



>* Distribution strategies scale training across many devices
>* They hide low-level details, enabling easy scaling

>* Unified interface for varied hardware distribution strategies
>* Supports GPUs, TPUs, and multi-machine training

>* Strategies plug into existing models and pipelines
>* Enable quick scaling from local experiments to clusters



In [ ]:
#@title Python Code - Distributed Training Strategies

# This script introduces TensorFlow distribution strategies.
# It uses a tiny model and synthetic data.
# It keeps output short and beginner friendly.

# !pip install tensorflow==2.20.0.

# Import standard libraries for reproducibility.
import os, random, math, sys

# Set deterministic random seeds for safety.
random.seed(7); os.environ["PYTHONHASHSEED"] = "7"

# Try importing TensorFlow inside a safe block.
try:
    import tensorflow as tf
except Exception as e:
    print("TensorFlow import failed, exiting safely.")
    sys.exit(0)

# Disable GPU to avoid CUDA handle errors in constrained environments.
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Check for available GPUs for potential distribution.
physical_gpus = tf.config.list_physical_devices("GPU")

# Decide whether to use MirroredStrategy or default.
if len(physical_gpus) > 1:
    strategy = tf.distribute.MirroredStrategy()
else:
    strategy = tf.distribute.get_strategy()

# Briefly report which strategy class is being used.
print("Using strategy:", strategy.__class__.__name__)

# Define tiny synthetic dataset parameters safely.
num_samples, num_features = 256, 8

# Create synthetic features as a small tensor.
features = tf.random.uniform((num_samples, num_features), seed=7)

# Create synthetic labels with a simple rule.
labels = tf.reduce_sum(features, axis=1, keepdims=True)

# Validate shapes before building dataset.
assert features.shape[0] == labels.shape[0]

# Build a small batched dataset for training.
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

# Shuffle and batch with a small buffer size.
dataset = dataset.shuffle(256, seed=7, reshuffle_each_iteration=False).batch(32)

# Create the model inside the strategy scope.
with strategy.scope():
    # Define a simple Sequential regression model.
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(num_features,)),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1)
    ])

# Compile the model with mean squared error loss.
with strategy.scope():
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss="mse",
        metrics=["mae"]
    )

# Train for a few epochs with silent logging.
history = model.fit(dataset, epochs=3, verbose=0)

# Evaluate the model briefly on a small batch.
small_batch = next(iter(dataset))

# Unpack the batch into inputs and targets.
x_batch, y_batch = small_batch

# Confirm batch shapes before evaluation.
print("Batch shape:", x_batch.shape, y_batch.shape)

# Evaluate without verbose logs for cleanliness.
loss, mae = model.evaluate(x_batch, y_batch, verbose=0)

# Show a concise summary of training quality.
print("Small batch loss:", float(loss))

# Show mean absolute error for interpretability.
print("Small batch MAE:", float(mae))

# Demonstrate a single prediction using the model.
sample_input = tf.expand_dims(x_batch[0], axis=0)

# Run prediction on the tiny sample input.
prediction = model.predict(sample_input, verbose=0)

# Print the true label and predicted value.
print("True label:", float(y_batch[0][0].numpy()), "Predicted:", float(prediction[0][0]))



## **3. TensorFlow API Landscape**

### **3.1. Stable and Deprecated APIs**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_C/image_03_01.jpg?v=1769364752" width="250">



>* Stable APIs signal long-term, backward-compatible support
>* Use them for reliable, scalable production ML systems

>* Deprecated APIs stay for now, but vanish later
>* Use them only while migrating to stable APIs

>* Stable APIs support long-term, maintainable ML projects
>* Deprecated APIs temporarily bridge legacy systems to modern



### **3.2. Model Export and Serving**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_C/image_03_02.jpg?v=1769364773" width="250">



>* SavedModel is the standard format for deployment
>* Supports TF Serving, TFLite, stable production use

>* Legacy frozen-graph and session workflows remain deprecated
>* Teams should migrate legacy systems to SavedModel workflows

>* Experimental export features enable cutting-edge deployment scenarios
>* Use them cautiously with version pinning and sandboxing



In [ ]:
#@title Python Code - Model Export and Serving

# This script introduces TensorFlow model export concepts.
# We simulate export and serving without external libraries.
# Focus is on stable legacy and experimental style APIs.

# Import standard libraries for paths and randomness.
import os
import json
import random

# Set deterministic random seed for reproducibility.
random.seed(42)

# Define a tiny fake model class for demonstration.
class TinyLinearModel:
    # Initialize with simple weight and bias parameters.
    def __init__(self, weight: float, bias: float):
        self.weight = float(weight)
        self.bias = float(bias)

    # Define a simple predict method like a model call.
    def predict(self, x_value: float) -> float:
        return self.weight * float(x_value) + self.bias

# Create a small instance representing a trained model.
stable_model = TinyLinearModel(weight=2.0, bias=0.5)

# Prepare a stable SavedModel style export dictionary.
stable_export = {
    "format": "savedmodel_like",
    "version": 1,
    "signatures": {"serving_default": {"inputs": ["x"], "outputs": ["y"]}},
    "weights": {"weight": stable_model.weight, "bias": stable_model.bias},
}

# Define a legacy frozen graph style export dictionary.
legacy_export = {
    "format": "frozen_graph_like",
    "graph_nodes": ["x", "mul", "add", "y"],
    "note": "represents older TensorFlow 1.x style graphs",
}

# Define an experimental export with extra quantization metadata.
experimental_export = {
    "format": "savedmodel_like_experimental",
    "version": 1,
    "quantization": {"scheme": "int8_per_channel", "status": "experimental"},
}

# Choose a small base directory for our fake exports.
base_dir = os.path.join(os.getcwd(), "tf_export_demo")

# Safely create the directory if it does not exist.
os.makedirs(base_dir, exist_ok=True)

# Helper function to save a dictionary as a JSON file.
def save_export_dict(export_dict: dict, file_path: str) -> None:
    # Validate that export_dict is not empty before saving.
    if not export_dict:
        raise ValueError("Export dictionary must not be empty.")

    # Write JSON with indentation for readability.
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(export_dict, file, indent=2)

# Save the stable export to a JSON file.
stable_path = os.path.join(base_dir, "stable_savedmodel_like.json")
save_export_dict(stable_export, stable_path)

# Save the legacy export to a JSON file.
legacy_path = os.path.join(base_dir, "legacy_frozen_graph_like.json")
save_export_dict(legacy_export, legacy_path)

# Save the experimental export to a JSON file.
experimental_path = os.path.join(base_dir, "experimental_savedmodel_like.json")
save_export_dict(experimental_export, experimental_path)

# Simulate loading a stable export and running a prediction.
with open(stable_path, "r", encoding="utf-8") as file:
    loaded_stable = json.load(file)

# Rebuild a TinyLinearModel instance from loaded weights.
weights = loaded_stable.get("weights", {})
reloaded_model = TinyLinearModel(weight=weights.get("weight", 1.0), bias=weights.get("bias", 0.0))

# Validate that the reloaded model behaves as expected.
input_value = 3.0
prediction = reloaded_model.predict(input_value)

# Collect short summary lines for printing.
summary_lines = [
    "Stable export path: " + os.path.basename(stable_path),
    "Legacy export path: " + os.path.basename(legacy_path),
    "Experimental export path: " + os.path.basename(experimental_path),
    "Stable format type: " + stable_export["format"],
    "Legacy format type: " + legacy_export["format"],
    "Experimental format type: " + experimental_export["format"],
    "Stable signature keys: " + ",".join(stable_export["signatures"].keys()),
    "Legacy graph nodes count: " + str(len(legacy_export["graph_nodes"])),
    "Experimental quantization scheme: " + experimental_export["quantization"]["scheme"],
    "Reloaded prediction for x=3.0: " + str(prediction),
]

# Print each summary line to illustrate key differences.
for line in summary_lines:
    print(line)




### **3.3. Experimental API Namespaces**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_C/image_03_03.jpg?v=1769364865" width="250">



>* Experimental APIs offer cutting-edge, not-yet-standard features
>* They may change or disappear; use cautiously

>* Experimental namespaces let users test new ideas
>* Successful features stabilize; weak ones change or disappear

>* Expect frequent changes; avoid mission-critical dependence
>* Use for research, gain insight into future features



# <font color="#418FDE" size="6.5" uppercase>**TF 2.20 Overview**</font>


In this lecture, you learned to:
- Summarize the core architectural concepts behind TensorFlow 2.x and eager execution. 
- Identify the primary high-level APIs in TensorFlow 2.20.0 and their roles. 
- Distinguish between stable, legacy, and experimental TensorFlow APIs relevant in 2026. 

In the next Module (Module 2), we will go over 'Tensor Basics'